In [1]:
import pandas as pd
import numpy as np

np.random.seed(1)

In [2]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.utils.np_utils import to_categorical
from keras.layers import Bidirectional

Using TensorFlow backend.


In [3]:
df = pd.read_csv('train_data/eng_train.csv', encoding='utf-8')

In [4]:
df.head(20)

,ids,sentences,OCR
0,1.1.1.1,oṁ namo bhagavate vāsudevāya,om namo bhagavate vasudevaya
1,1.1.2.1,dharmaḥ projjhita-kaitavo ’tra paramo nirmatsa...,dharmah projjhita-kaitavo 'tra paramo nirmatsa...
2,1.1.2.2,vedyaṁ vāstavam atra vastu śivadaṁ tāpa-trayon...,vedyam vastavam atra vastu sivadam tapa-trayon...
3,1.1.2.3,śrīmad-bhāgavate mahā-muni-kṛte kiṁ vā parair ...,Ssrimad-bhagavate maha-muni-krte kim va parair...
4,1.1.2.4,sadyo hṛdy avarudhyate ’tra kṛtibhiḥ śuśrūṣubh...,sadyo hrdy avarudhyate 'tra krtibhih sugsrusub...
5,1.1.3.1,nigama-kalpa-taror galitaṁ phalaṁ,nigama-kalpa-taror galitam phalam
6,1.1.3.2,śuka-mukhād amṛta-drava-saṁyutam,suka-mukhad amrta-drava-samyutam
7,1.1.3.3,pibata bhāgavataṁ rasam ālayam,pibata bhagavatam rasam alayam
8,1.1.3.4,muhur aho rasikā bhuvi bhāvukāḥ,muhur aho rasika bhuvi bhavukah
9,1.1.4.1,naimiṣe ’nimiṣa-kṣetre,naimise 'nimisa-ksetre


In [5]:
df.describe()

,ids,sentences,OCR
count,50980,50980,50971
unique,45781,43891,43843
top,4.14.7.3,śrī-śuka uvāca,Sri-Suka uvaca
freq,2,346,346


In [6]:
filter_values = df['sentences'].str.len() == df['OCR'].str.len()

In [7]:
len(filter_values)

50980

In [8]:
filtered_df = df[filter_values]

In [9]:
filtered_df.head()

,ids,sentences,OCR
0,1.1.1.1,oṁ namo bhagavate vāsudevāya,om namo bhagavate vasudevaya
1,1.1.2.1,dharmaḥ projjhita-kaitavo ’tra paramo nirmatsa...,dharmah projjhita-kaitavo 'tra paramo nirmatsa...
2,1.1.2.2,vedyaṁ vāstavam atra vastu śivadaṁ tāpa-trayon...,vedyam vastavam atra vastu sivadam tapa-trayon...
5,1.1.3.1,nigama-kalpa-taror galitaṁ phalaṁ,nigama-kalpa-taror galitam phalam
6,1.1.3.2,śuka-mukhād amṛta-drava-saṁyutam,suka-mukhad amrta-drava-samyutam


In [10]:
len(filtered_df) / len(df)

0.9351510396233818

In [11]:
filtered_df['OCR'] = filtered_df['OCR'].astype(str).str.lower()

/Users/rbhat/anaconda2/envs/san_ocr_enc_dec/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
filtered_df['OCR'].head()

0                         om namo bhagavate vasudevaya
1    dharmah projjhita-kaitavo 'tra paramo nirmatsa...
2    vedyam vastavam atra vastu sivadam tapa-trayon...
5                    nigama-kalpa-taror galitam phalam
6                     suka-mukhad amrta-drava-samyutam
Name: OCR, dtype: object

In [13]:
vocab = set(filtered_df['sentences'].str.cat() + filtered_df['OCR'].str.cat())

In [14]:
string_lengths = sorted(filtered_df['OCR'].str.len())

In [15]:
SEQ_LENGTH = int(np.percentile(sorted(string_lengths), 99.5))
VOCAB_SIZE = 56

In [36]:
SEQ_LENGTH = 50

In [37]:
vocab = sorted(vocab)
vocab_to_int = {c: i+1 for i, c in enumerate(vocab)}
int_to_vocab = {i+1:c for i,c in enumerate(vocab)}

In [38]:
int_to_vocab[1]

' '

In [39]:
X = filtered_df['OCR'].tolist()

In [40]:
Y = filtered_df['sentences'].tolist()

In [41]:
print(X[0])
print(Y[0])

om namo bhagavate vasudevaya
oṁ namo bhagavate vāsudevāya


In [42]:
X_train = []
Y_train = []

for x,y in zip(X, Y):
    
    X_train.append(np.array([vocab_to_int[ch] for ch in x]))
    Y_train.append(np.array([vocab_to_int[ch] for ch in y]))

In [43]:
X_train = np.array(X_train)
X_train[0]

array([26, 24,  1, 25, 12, 24, 26,  1, 13, 19, 12, 18, 12, 33, 12, 31, 16,
        1, 33, 12, 30, 32, 15, 16, 33, 12, 34, 12])

In [44]:
X_train = pad_sequences(X_train, maxlen=SEQ_LENGTH, dtype='int32', padding='post', truncating='post')
Y_train = pad_sequences(Y_train, maxlen=SEQ_LENGTH, dtype='int32', padding='post', truncating='post')

In [45]:
X_train[0]

array([26, 24,  1, 25, 12, 24, 26,  1, 13, 19, 12, 18, 12, 33, 12, 31, 16,
        1, 33, 12, 30, 32, 15, 16, 33, 12, 34, 12,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [46]:
Y_train = to_categorical(Y_train)

In [47]:
print(X_train.shape)
print(Y_train.shape)

(47674, 50)
(47674, 50, 56)


In [48]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, 50, input_length=SEQ_LENGTH))
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            2800      
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 64)            29440     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 50, 56)            3640      
_________________________________________________________________
activation_2 (Activation)    (None, 50, 56)            0         
Total params: 35,880
Trainable params: 35,880
Non-trainable params: 0
_________________________________________________________________
None


In [49]:
model.fit(X_train, Y_train, batch_size=512, verbose=1, epochs=20, validation_split=0.05)

Train on 45290 samples, validate on 2384 samples
Epoch 1/20
45290/45290 [==============================] - 34s 747us/step - loss: 2.4168 - acc: 0.5083 - val_loss: 1.7862 - val_acc: 0.5482
Epoch 2/20
45290/45290 [==============================] - 34s 757us/step - loss: 1.6761 - acc: 0.5885 - val_loss: 1.5534 - val_acc: 0.6034
Epoch 3/20
45290/45290 [==============================] - 33s 732us/step - loss: 1.4547 - acc: 0.6109 - val_loss: 1.2200 - val_acc: 0.6734
Epoch 4/20
45290/45290 [==============================] - 33s 719us/step - loss: 0.8800 - acc: 0.8116 - val_loss: 0.5354 - val_acc: 0.9025
Epoch 5/20
45290/45290 [==============================] - 32s 715us/step - loss: 0.3860 - acc: 0.9182 - val_loss: 0.2779 - val_acc: 0.9238
Epoch 6/20
45290/45290 [==============================] - 32s 716us/step - loss: 0.2436 - acc: 0.9274 - val_loss: 0.2129 - val_acc: 0.9300
Epoch 7/20
45290/45290 [==============================] - 34s 740us/step - loss: 0.2009 - acc: 0.9313 - val_loss: 0.1

In [50]:
prediction = np.argmax(model.predict(X_train[:1]), axis=2)
prediction[0]

array([26, 24,  1, 25, 12, 24, 26,  1, 13, 19, 12, 18, 12, 33, 12, 31, 16,
        1, 33, 12, 30, 32, 15, 16, 33, 12, 34, 12,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [52]:
prediction_string = "".join([int_to_vocab[c] if c > 0 else '' for c in prediction[0]])
print(prediction_string)

om namo bhagavate vasudevaya


In [53]:
X_train[1], np.argmax(Y_train[1], axis=1)

(array([15, 19, 12, 29, 24, 12, 19,  1, 27, 29, 26, 21, 21, 19, 20, 31, 12,
         7, 22, 12, 20, 31, 12, 33, 26,  1,  5, 31, 29, 12,  1, 27, 12, 29,
        12, 24, 26,  1, 25, 20, 29, 24, 12, 31, 30, 12, 29, 12, 25, 12],
       dtype=int32),
 array([15, 19, 12, 29, 24, 12, 45,  1, 27, 29, 26, 21, 21, 19, 20, 31, 12,
         7, 22, 12, 20, 31, 12, 33, 26,  1, 55, 31, 29, 12,  1, 27, 12, 29,
        12, 24, 26,  1, 25, 20, 29, 24, 12, 31, 30, 12, 29, 40, 50, 40]))

In [56]:
X[0]

'om namo bhagavate vasudevaya'

In [57]:
Y[0]

'oṁ namo bhagavate vāsudevāya'

In [58]:
!ls

README.md                 for_all_cantos.sh         tesseract_ocr.sh
bhagabatam_dataprep.ipynb results_bilstm.csv        test_data
cantos                    seq2seq-lstm.ipynb        text2img
cantos_crawl.py           seq2seq.ipynb             train_data


In [59]:
test_df = pd.read_csv('test_data/test_data_combined.csv', encoding='utf-8')

In [60]:
test_df.head()

,source,target,origin
0,dhrtarastra uvaca,dhṛtarāṣṭra uvāca,pdf
1,dharma-ksetre kuru-ksetre samaveta yuyutsavah,dharma-kṣetre kuru-kṣetre samavetā yuyutsavaḥ,pdf
2,mamakah pandavas caiva kim akurvata sanjaya,māmakāḥ pāṇḍavāś caiva kim akurvata sañjaya,pdf
3,karpanya-dosopahata-svabhavah,kārpaṇya-doṣopahata-svabhāvaḥ,pdf
4,prechami tvam dharma-sammudha-cetah,pṛcchāmi tvāṁ dharma-sammūḍha-cetāḥ,pdf


In [61]:
filter_testdf = test_df['source'].str.len() == test_df['target'].str.len()

In [62]:
filter_testdf.head()

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [63]:
filter_testdf =  test_df[filter_testdf]

In [64]:
filter_testdf.head()

,source,target,origin
0,dhrtarastra uvaca,dhṛtarāṣṭra uvāca,pdf
1,dharma-ksetre kuru-ksetre samaveta yuyutsavah,dharma-kṣetre kuru-kṣetre samavetā yuyutsavaḥ,pdf
2,mamakah pandavas caiva kim akurvata sanjaya,māmakāḥ pāṇḍavāś caiva kim akurvata sañjaya,pdf
3,karpanya-dosopahata-svabhavah,kārpaṇya-doṣopahata-svabhāvaḥ,pdf
4,prechami tvam dharma-sammudha-cetah,pṛcchāmi tvāṁ dharma-sammūḍha-cetāḥ,pdf


In [65]:
filter_testdf['source'] = filter_testdf['source'].astype(str).str.lower()

/Users/rbhat/anaconda2/envs/san_ocr_enc_dec/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [66]:
X_test = []
for x in filter_testdf['source'].tolist():    
    X_test.append(np.array([vocab_to_int.get(ch, 0) for ch in x]))



In [67]:
X_test[:2]

[array([15, 19, 29, 31, 12, 29, 12, 30, 31, 29, 12,  1, 32, 33, 12, 14, 12]),
 array([15, 19, 12, 29, 24, 12,  7, 22, 30, 16, 31, 29, 16,  1, 22, 32, 29,
        32,  7, 22, 30, 16, 31, 29, 16,  1, 30, 12, 24, 12, 33, 16, 31, 12,
         1, 34, 32, 34, 32, 31, 30, 12, 33, 12, 19])]

In [68]:
X_test = pad_sequences(X_test, maxlen=SEQ_LENGTH, dtype='int32', padding='post', truncating='post')

In [73]:
x_test_pred = model.predict_classes(X_test, verbose=1)

299/299 [==============================] - 0s 323us/step


In [74]:
x_test_pred

array([[15, 19, 51, ...,  0,  0,  0],
       [15, 19, 12, ...,  0,  0,  0],
       [24, 12, 24, ...,  0,  0,  0],
       ...,
       [30, 12, 29, ...,  0,  0,  0],
       [24, 12, 19, ...,  0,  0,  0],
       [14, 32, 15, ...,  0,  0,  0]])

In [75]:
test_prediction_string =[]
for row in x_test_pred:
    test_prediction_string.append("".join([int_to_vocab[c] if c > 0 else '' for c in row]))

In [76]:
test_prediction_string

['dhṛtarastra uvāca',
 'dharma-kṣetre kuru-kṣetre samaveta yuyutsavaḥ',
 'mamakaḥ pandavas caiva kim akurvata sanjaya',
 'karpaṇya-doṣopaḥata-svabhavaḥ',
 'prechaṁi tvaṁ dharma-sammudha-cetaḥ',
 'yac chreyaḥ syan niścitaṁ bruhi tan me',
 'siṣyas te ’haṁ sadhi mam tvaṁ prapannam',
 'srī-bhagavan uvaca',
 'asocyan anvasocas tvaṁ prajña-vadaṁs ca bhasase',
 'gatasun agatasums ca nanuṣocanti panditaḥ',
 'na tv evaḥaṁ jatu nasam na tvam neme janadhipaḥ',
 'na caiva na bhaviṣyamaḥ sarve vayaṁ ataḥ param',
 'dehino ’smin yatha dehe kaumaraṁ yauvanaṁ jara',
 'tathā dehantara-praptir dhiras tatra na muhyati',
 'matra-sparṣas tu kaunteya sitoṣṇa-sukha-duḥkha-dah',
 'agamapayino ’nityas tams titikṣasva bharata',
 'avināsi tu tad viddhi yena sarvaṁ idam tatam',
 'vināsaṁ avyayasyasya na kascit kartum arhati',
 'na jayate mṛiyate va kadacin nayam bhūtva bhavita ',
 'ajo nityaḥ sasvato ’yam puraṇo na hanyate hanyaṁan',
 'vasaṁsi jirṇani yatha vihāya',
 'navani gṛhṇati naro ’paraṇi',
 'tathā sariraṇi

In [79]:
filter_testdf['prediction'] = test_prediction_string

/Users/rbhat/anaconda2/envs/san_ocr_enc_dec/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [80]:
filter_testdf.head()

,source,target,origin,prediction
0,dhrtarastra uvaca,dhṛtarāṣṭra uvāca,pdf,dhṛtarastra uvāca
1,dharma-ksetre kuru-ksetre samaveta yuyutsavah,dharma-kṣetre kuru-kṣetre samavetā yuyutsavaḥ,pdf,dharma-kṣetre kuru-kṣetre samaveta yuyutsavaḥ
2,mamakah pandavas caiva kim akurvata sanjaya,māmakāḥ pāṇḍavāś caiva kim akurvata sañjaya,pdf,mamakaḥ pandavas caiva kim akurvata sanjaya
3,karpanya-dosopahata-svabhavah,kārpaṇya-doṣopahata-svabhāvaḥ,pdf,karpaṇya-doṣopaḥata-svabhavaḥ
4,prechami tvam dharma-sammudha-cetah,pṛcchāmi tvāṁ dharma-sammūḍha-cetāḥ,pdf,prechaṁi tvaṁ dharma-sammudha-cetaḥ


In [81]:
filter_testdf.to_csv('results_lstm.csv', encoding='utf-8')